In [1]:
import numpy as np
import pandas as pd

path_to_csv = "../data/mortgage_transformed.csv"
df = pd.read_csv(path_to_csv)
df.set_index(df.pop("Unnamed: 0"), inplace=True)

X = df.dropna(axis=0)
y = X.pop("result")

# A model that incorporates what we've learned thus far

We will use most features from the original datasets, while adding new ones. We will also try to normalize the data to improve model performance

In [59]:
# custom features
X["time_since_start"] = X.time - X.orig_time
X["time_until_end"] = X.mat_time - X.time
X["mortgage_duration"] = X.mat_time - X.orig_time
X["balance_change"] = X.balance_time-X.balance_orig_time
X["LTV_change"] = X.LTV_time - X.LTV_orig_time
X["interest_change"] = X.interest_rate_time - X.Interest_Rate_orig_time
# X["interest_change_as_fraction"] = X.interest_change / X.Interest_Rate_orig_time
X["hpi_change"] = X.hpi_time - X.hpi_orig_time

In [60]:
# select useful columns
used_columns = [
    "time_since_start", "time_until_end", "mortgage_duration",
    "balance_time", "balance_orig_time", "balance_change",
    "LTV_time", "LTV_orig_time", "LTV_change",
    "interest_rate_time", "Interest_Rate_orig_time", "interest_change",
    #"interest_change_as_fraction",
    "hpi_time", "hpi_orig_time",
    "hpi_change", "uer_time"
]

In [75]:
# normalize the data
from sklearn.preprocessing import normalize
X_data = X.loc[:, used_columns]
X_norm = normalize(X_data)

In [83]:
# split the data for testing the model
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X_data, y,
    train_size=0.7,
    test_size=0.3)

In [79]:
# we will try to select a better model using accuracy scoring
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

def score_model(X_train, y_train, X_valid, y_valid, n_estimators, learning_rate=0.1):
    model = XGBClassifier(
                n_estimators = n_estimators,
                learning_rate = learning_rate,
                n_jobs = 5,
                eval_metric = 'mlogloss'
            )
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    
    accuracy = accuracy_score(predictions, y_valid)
    return accuracy

In [82]:
# shut up warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# we will try different number of estimators to see, which is the best fit
estimator_count = [10, 25, 36, 50, 100]

last_accuracy = 0
for n_estimators in estimator_count:
    accuracy = score_model(
        X_train, y_train, X_valid, y_valid,
        n_estimators=n_estimators,
        learning_rate=0.5 # let's speed things up
    )
    print(f"The accuracy with n_estimators is {n_estimators} with an accuracy of {accuracy}")

The accuracy with n_estimators is 10 with an accuracy of 0.6596495095337815
The accuracy with n_estimators is 25 with an accuracy of 0.6605312465557147
The accuracy with n_estimators is 36 with an accuracy of 0.6578309269260443
The accuracy with n_estimators is 50 with an accuracy of 0.6552959329879864
The accuracy with n_estimators is 100 with an accuracy of 0.6543039788383115


In [164]:
some_model = XGBClassifier(
    n_estimators = 24,
    learning_rate = 0.3,
    reg_lambda=0.9,
    reg_alpha=0.1,
    objective='reg:logistic',
    eval_metric = 'error',
)
some_model.fit(X_train, y_train)
predictions = some_model.predict(X_valid)
accuracy = accuracy_score(predictions, y_valid)
print(f"Accuracy is {accuracy}")

Accuracy is 0.6707814394356884


In [158]:
X.shape

(60484, 32)

In [170]:
train_predictions = some_model.predict(X_train)
t_accuracy = accuracy_score(train_predictions, y_train)
print(f"Train accuracy is {t_accuracy}")

Train accuracy is 0.6999149700033067


In [165]:
def head_to_head(preds, values):
    total = len(preds)
    correct = 0
    for pred, value in zip(preds, values):
        if pred == value:
            correct += 1
    return correct/total

In [168]:
head_to_head(predictions, y_valid)

0.6707814394356884

In [171]:
preds = some_model.predict_proba(X_train)

In [172]:
print(preds)

[[0.81363994 0.07159667 0.11476336]
 [0.7206438  0.16781428 0.11154196]
 [0.8355657  0.01496294 0.14947145]
 ...
 [0.511739   0.37817734 0.11008359]
 [0.9516614  0.02887007 0.01946845]
 [0.7185148  0.06138965 0.22009552]]


,id,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,...,status_time,sample,time_since_start,time_until_end,mortgage_duration,balance_change,LTV_change,interest_change,interest_change_as_fraction,hpi_change
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,1,32,-7,25,113,35877.03,25.872559,9.200,186.12,1.104163,...,0.0,public,39,81,120,-9122.97,-43.527441,0.000,0.000000,99.09
1,1,40,-7,25,113,32388.30,27.828850,9.200,156.21,1.585966,...,0.0,public,47,73,120,-12611.70,-41.571150,0.000,0.000000,69.18
2,4,28,-2,25,119,60576.14,34.672545,10.875,219.67,1.229172,...,0.0,public,30,91,121,-3173.86,-47.127455,0.375,0.035714,121.68
3,4,42,-2,25,119,57950.27,45.256864,9.250,161.00,3.029499,...,0.0,public,44,77,121,-5799.73,-36.543136,-1.250,-0.119048,63.01
4,4,52,-2,25,119,55332.84,43.177891,10.500,161.13,1.081049,...,0.0,public,54,67,121,-8417.16,-38.622109,0.000,0.000000,63.14
5,5,26,18,25,138,52199.00,65.671251,9.155,225.10,2.151365,...,0.0,public,8,112,120,-601.00,-14.328749,0.000,0.000000,38.19
6,6,29,19,25,139,206802.52,68.722296,9.580,217.37,1.692969,...,0.0,public,10,110,120,-5197.48,-11.277704,2.980,0.451515,25.95
7,6,37,19,25,139,203769.85,95.983721,9.500,153.35,-4.146711,...,0.0,public,18,102,120,-8230.15,15.983721,2.920,0.443769,-38.07
8,6,49,19,25,139,194630.73,90.738020,6.580,154.94,2.456634,...,0.0,public,30,90,120,-17369.27,10.738020,0.000,0.000000,-36.48
